In [2]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import torch
import wandb
torch.cuda.empty_cache()

# Load the model and tokenizer
model_name = "MBZUAI-Paris/Atlas-Chat-2B"
tokenizer = AutoTokenizer.from_pretrained(model_name,padding_side='right')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.25it/s]


In [3]:
model

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=

In [1]:
import dask.dataframe as dd

df = dd.read_parquet("hf://datasets/MBZUAI-Paris/Darija-SFT-Mixture/data/train-*.parquet")

/home/jihad.rbaiti/lustre/pt_cloud-muhqxqc6fxo/users/jihad.rbaiti/conda/xalma/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
df

,dataset,id,messages,direction,metadata
npartitions=2,,,,,
,string,string,object,string,object
,...,...,...,...,...
,...,...,...,...,...


In [2]:
'''from datasets import load_dataset

ds = load_dataset("MBZUAI-Paris/Darija-SFT-Mixture")'''

'from datasets import load_dataset\n\nds = load_dataset("MBZUAI-Paris/Darija-SFT-Mixture")'

In [3]:
message_example = df['messages'].head(1)


In [7]:
message_example

0    [{'content': 'ترجم من الدارجة للفصحى:
ماكايعجب...
Name: messages, dtype: object

In [10]:
a = message_example.iloc[0]

In [26]:
import pandas as pd
def extract_content(messages, role):
    return [msg['content'] for msg in messages if msg['role'] == role]

new_df = pd.DataFrame()
new_df['user_content'] = df['messages'].map(lambda x: extract_content(x, 'user'), meta=('x', 'object'))
new_df['assistant_content'] = df['messages'].map(lambda x: extract_content(x, 'assistant'), meta=('x', 'object'))

In [38]:
new_df['user_content'].iloc[1]

['كمل هاد لقصة:\nشراف كيشاف خيال ديال شي واحد واقف حدا لباب...فتحو طار على هداك لي فتح لباب 👊جمع يدو عيضربو\xa0\nحسااام:هااااااو بغي تفركتلي ضروسي راه غير انا يحاول يطلال من براكة\xa0\nشراف:عتهبط عينينك ولا بربي نجبدهم لمك تلعب بيهم البي ارا حوايج لي جبتي وبقا هنا مدخالش\xa0\nحسام:وخا ومال هدووو طحطحتيهم هنا 👀\nشراف:دخل ورد الباب كلس حدا بيان كيقلب راسها لقاها طايبه لبسها 2بياس سروال دجين فابيض مقطع مع تيشورط معري من يدين بيض وخلالها شعرها مطلوق كيحاول يجمعو كيلقاه فازك شي شويه من جيهت جبهتها وتحت راسها هزها بين يديه قربها لصدرو وشعرها طايح ودفع الباب برجليه وتم غادي\nحسام:واااه واااه مشاء الله ومنين جا هاد الخير.....\nشراف:(ضار عندو وعينيه غيخرجو من بلاصتهم)😠😠😠متخلينيش نسا راك صاحبي\xa0\nحسام:🙊🙊🙊هاهو ساكت\xa0\nشراف:تم غادي بيها فطريق الغابه توصل طنوبيل فتحلو حسام وحطها فحجرو طير اتبي خلاص منبغبكش تعطل\xa0\nحسام:كسيرا 160عفطه وحده وفين عتمشي دب\nشراف:دينا لووووطيل\xa0\nحسام:واخا ولكن خاصنا ندويو عندنا مهمه رسميه خاصنا على أقل منهنا 3ايام نكونو\xa0\nشراف(قاطعو)ماشي وقتو دب عندي شغل دب ن

In [42]:
import time

for i in range(7):
    print(i,': ')
    print(new_df['assistant_content'].iloc[i][0])
    print()
    time.sleep(10)

0 : 
أنا لا أحب القصص المصورة.

1 : 
خرج من لوطو وتم غادي وقف فريسبشن 
شراف:عطيني ساروت دابا 
....واخا مسيو هي لمتعاودش
طلع حطها فوق نموسيه وغطاها وحرارتها مازاله مرتافعه هبط لحنبيكاتها لي مزنكين بحراره كيدوز عليهم بيديه تسمع دقان وناض حل 
حسام:ها طبيبة
طبيةبغات تسلم وسي شراف وجه ليها قصف ثلاتي ابعاد
شراف:حيدي عليا من تسلام وشوفي شغلاك (شاف فحسام) ونت خرج وانت واش واش عنباتو هنا سربينا خلاص
دكتوره:(كتفتف)اه..اه..قاستها لحراره لقاتها طالعه بزاف ودربتلها بره وكتبات ليها فورة دوا
شراف:شنو سباب ديال حراره 
دكتوره: غير ضربها شوية دالبرد وباخص كتبالي جسمها معنف بزاف وضعيف داكشي لاش ضغية مرضات خاصها راحة تامة(مدات ليه دوا)خاصها تشرب هاد دوا ضاروري
شرف(شد الورقة)صافي تقدري تمشي
راقب شراف طبيبة حتى خرجات ورجع عند بيان غطاها مزيان وباسها فجبهتها وخرج...هبط البار دؤطيل حيت عارف حسام غيكون غير تما قشعو من بعيد شاد يدي بارميطة كيتعنب عليها 
شراف(كلس حداه)فودكا
...(شافتو ثوثرات)واخا مسيو 
حسام:كنكون مكونسونطري تاكيجي با شراف كيدورونجينا
شراف:كولشي شكاين بلا كثرة الهضرة(حطات بارميطة كاس لشراف)فين هاد

In [81]:
new_df

,user_content,assistant_content
0,[ترجم من الدارجة للفصحى:\nماكايعجبونيش لكتوبا ...,[أنا لا أحب القصص المصورة.]
1,[كمل هاد لقصة:\nشراف كيشاف خيال ديال شي واحد و...,[خرج من لوطو وتم غادي وقف فريسبشن \nشراف:عطيني...
2,[لخص هاد المقطع:\n تواصلَ إرتفاع عدد الإصابات...,[الداخلة .. 19 تقاسو بفيروس كورونا ]
3,[ترجم من الفصحى للدارجة:\nإنه يزعجني],[كايبرزطني]
4,[هادا سؤال متعدد الخيارات (مع الجواب ديالو) عل...,[A]
...,...,...
458280,[شنو هو الإحساس ديال هاد الجملة؟\nالعبارة: اخر...,[ماكينش إحساس]
458281,[شنو هو الإحساس ديال هاد الجملة؟\nالعبارة: : ن...,[ماكينش إحساس]
458282,[ترجم من الدارجة للفرنساوية:\nهادي مسريعاش بزا...,"[Ces derniers ne sont pas très agiles, de sort..."
458283,[ترجم من الدارجة للإنجليزية:\nالسبب الرئيسي نت...,[The main reason for this phenomenon is the re...


In [52]:
import dask.dataframe as dd

# Assume df_new already has the columns 'user_content_translated' and 'assistant_content_translated'
# Use a lambda function to convert list to a single string
new_df1 = pd.DataFrame()
new_df1['user_content'] = new_df['user_content'].map(lambda x: ' '.join(x) if x else '')
new_df1['assistant_content'] = new_df['assistant_content'].map(lambda x: ' '.join(x) if x else '')


In [83]:
new_df1.to_csv('/home/jihad.rbaiti/Work_2/CPO/data/QuestionAnswering/data_QA.csv', index=False) 

In [7]:
import pandas as pd

data = pd.read_csv('/home/jihad.rbaiti/Work_2/CPO/data/QuestionAnswering/data_QA.csv')

In [12]:
from sklearn.model_selection import train_test_split
train_val_data, test_data = train_test_split(data, test_size=0.8, random_state=42)
train_data, val_data = train_test_split(train_val_data, test_size=0.05, random_state=42) 


print('Whole data length', len(data))
print("Training set size:", len(train_data),'(', len(train_data)/len(data)*100,'% )')
print("Validation set size:", len(val_data),'(', len(val_data)/len(data)*100,'% )')
print("Test set size:", len(test_data),'(',len(test_data)/len(data)*100,'% )')


Whole data length 458285
Training set size: 87074 ( 18.99996726927567 % )
Validation set size: 4583 ( 1.0000327307243309 % )
Test set size: 366628 ( 80.0 % )


In [14]:
train_data.to_csv('/home/jihad.rbaiti/Work_2/CPO/data/QuestionAnswering/train.csv', index=False)
val_data.to_csv('/home/jihad.rbaiti/Work_2/CPO/data/QuestionAnswering/val.csv', index=False)
test_data.to_csv('/home/jihad.rbaiti/Work_2/CPO/data/QuestionAnswering/test.csv', index=False)


In [15]:
val_data[:100].to_csv('/home/jihad.rbaiti/Work_2/CPO/data/QuestionAnswering/sample100.csv', index=False)

In [2]:
data['assistant_content'].iloc[185]

nan

In [79]:
new_df1[['assistant_content']].iloc[36100]

assistant_content    hoa mirikani
Name: 36100, dtype: object

In [80]:
new_df['assistant_content'].iloc[36100]

['hoa mirikani']

In [23]:

new_df['assistant_content'].iloc[0][0]
for i in range(len(new_df['assistant_content'])):
    new_df['assistant_content'].iloc[0] = new_df['assistant_content'].iloc[0][0]

In [11]:
type(a)

numpy.ndarray

In [13]:
a[0]['content']

'ترجم من الدارجة للفصحى:\nماكايعجبونيش لكتوبا ديال أرّوسّوم.'

In [9]:
print(message_example.iloc[0])


[{'content': 'ترجم من الدارجة للفصحى:\nماكايعجبونيش لكتوبا ديال أرّوسّوم.', 'role': 'user'}
 {'content': 'أنا لا أحب القصص المصورة.', 'role': 'assistant'}]


In [4]:
df = df.repartition(npartitions=10)

In [5]:
df = df.persist()

In [10]:
df = df.set_index('id')

In [12]:
selected_row = df.partitions[1].iloc[0].compute()


NotImplementedError: 'DataFrame.iloc' only supports selecting columns. It must be used like 'df.iloc[:, column_indexer]'.

In [8]:
df.sample(frac=0.01)

,dataset,id,messages,direction,metadata
npartitions=10,,,,,
,string,string,object,string,object
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [7]:
df[:10].to_csv('test.csv', index=False)

NotImplementedError: 'DataFrame.iloc' only supports selecting columns. It must be used like 'df.iloc[:, column_indexer]'.